In [ ]:
#!/usr/bin/env python

import sys
sys.path.append('/home/bij/Projects/fdtd/')
import fdtd
import fdtd.backend as bd
import matplotlib.pyplot as plt


# ## Set Backend
#fdtd.set_backend("numpy")
fdtd.set_backend("torch")


# ## Constants
WAVELENGTH = 1550e-9
WAVELENGTH2 = 1550e-8
SPEED_LIGHT: float = 299_792_458.0  # [m/s] speed of light


# ## Simulation

# create FDTD Grid

# In[4]:


grid = fdtd.Grid(
    (1.5e-5, 1.5e-5, 1),
    grid_spacing=0.1 * WAVELENGTH,
    permittivity=1.0,
    permeability=1.0,
)

print('Grid Shape: ', grid.shape)


# boundaries

# In[5]:


# grid[0, :, :] = fdtd.PeriodicBoundary(name="xbounds")
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")

# grid[:, 0, :] = fdtd.PeriodicBoundary(name="ybounds")
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")

grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")


# sources

grid[20, 25, 0] = fdtd.PointSource(
    period=WAVELENGTH / SPEED_LIGHT, name="linesource0", amplitude=1,
)
#grid[20, -25, 0] = fdtd.PointSource(
#    period=WAVELENGTH / SPEED_LIGHT, name="linesource1", pulse=True, cycle=5193414489032258,
#)
grid[20, -25, 0] = fdtd.PointSource(
    period=WAVELENGTH / SPEED_LIGHT, name="linesource1", delay=10,
)


# detectors

# grid[12e-6, :, 0] = fdtd.LineDetector(name="detector")


# objects

# ## Run simulation

grid_E_energies = None
grid.visualize(z=0, animate=True)
for i in range(1000):
    grid.run(1, progress_bar=False)
    grid_energy_E, grid_energy_H = grid.visualize(z=0, norm='linear', animate=True)
    if(i == 0):
        grid_E_energies = grid_energy_E
    else:
        grid_E_energies += grid_energy_E
    plt.show()


In [ ]:
grid_E_energies

In [ ]:
plt.imshow(bd.numpy(grid_E_energies))
plt.savefig('foo.png')


In [ ]:
for shift in range(0, 14):
    grid = fdtd.Grid(
        (1.5e-5, 1.5e-5, 1),
        grid_spacing=0.1 * WAVELENGTH,
        permittivity=1.0,
        permeability=1.0,
    )
    grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
    grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
    grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
    grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
    grid[:, :, 0] = fdtd.PeriodicBoundary(name="zbounds")
    
    y_mid, x_mid = grid.shape[0]//2, grid.shape[1]//2
    
    grid[y_mid, 25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource0", amplitude=1,
    )
    grid[y_mid, -25, 0] = fdtd.PointSource(
        period=WAVELENGTH / SPEED_LIGHT, name="linesource1", delay=shift,
    )

    grid_E_energies = None
    grid.visualize(z=0, animate=True)
    for i in range(100):
        grid.run(1, progress_bar=False)
        grid_energy_E, grid_energy_H = grid.visualize(z=0, norm='linear', animate=True)
        if(i == 0):
            grid_E_energies = grid_energy_E
        else:
            grid_E_energies += grid_energy_E
        #plt.show()
    plt.imshow(bd.numpy(grid_E_energies))
    plt.savefig('avg_energy_shift_mid={0}.png'.format(shift))
